<a href="https://colab.research.google.com/github/nonoumasy/LA-County-Covid-Cases/blob/master/LA_County_Covid_Cases_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install gspread oauth2client df2gspread

In [0]:
import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim

import gspread 
from oauth2client.service_account import ServiceAccountCredentials
from df2gspread import df2gspread as d2g

In [0]:
# load data from url
df = pd.DataFrame(pd.read_html('http://publichealth.lacounty.gov/media/Coronavirus/locations.htm')[0])

In [0]:
# skip rows
la_df = df.iloc[28:, :].reset_index(drop=True)

# long beach and pasadena
lb_df = df.iloc[2:4, :]

deaths_df = df.iloc[[0, 5], 0:2]
# rename to 'cases' 
deaths_df.iloc[0, 0] = deaths_df.iloc[0, 0].replace('Laboratory Confirmed Cases (LCC)', 'Cases')

In [0]:
def cleanup_df(df):
  df.columns = df.columns.str.lower().str.replace(' ', '_')
  df.columns = ['locations', 'total_cases', 'rate']

  # cleanup values
  df.locations = df.locations.str.replace('*', '').str.replace('City of ', '').str.replace('Los Angeles - ', '').str.replace('Unincorporated - ', '').str.replace('-', '').str.strip()
  df['total_cases'] = df.total_cases.str.replace('--', '0')
  df['rate'] = df.rate.str.replace('--', '0')

  # convert to numeric column
  df['total_cases'] = pd.to_numeric(df['total_cases'])
  df['rate'] = pd.to_numeric(df['rate'])

  # drop some rows
  df = df[df.locations != 'Los Angeles']
  df = df[df.locations != 'Under Investigation']

  # split location and keep first value
  #df['locations'] = df['locations'].apply(lambda x : x.split('/')[0])

  #
  df.total_cases.replace(0, np.nan, inplace=True)
  df.dropna(subset=['total_cases'], inplace=True)
  df.drop_duplicates('locations', inplace=True)
  #
  df = df.sort_values('locations')

  # reset index
  df.reset_index(drop=True, inplace=True)

  return df

In [36]:
la_df = cleanup_df(la_df)
lb_df = cleanup_df(lb_df)

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [0]:
# remember to mount Google drive
json_path = '/content/drive/My Drive/jupyter-275007-0fd4fe387056.json'

# Connect to our service account and open Google Sheet
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
gc = gspread.authorize(credentials)

spreadsheet_key = '1S7kIiYZlNIROuj00H_qOv1posb8kec_yjk2aIz4QHeM' 
book = gc.open_by_key(spreadsheet_key) 
worksheet = book.worksheet('locations') 
table = worksheet.get_all_values()

# Convert table data into a dataframe
locations_df = pd.DataFrame(table[1:], columns=table[0])

In [0]:
# merge la and lbf
new_df = pd.concat([la_df, lb_df], ignore_index=True)

#new_df.drop_duplicates('locations', keep='last', inplace=True)
new_df.reset_index(drop=True, inplace=True)

# merge with geocoded locations
temp_df = new_df.merge(locations_df)

In [39]:
# save to Google Sheets

spreadsheet_key = '1S7kIiYZlNIROuj00H_qOv1posb8kec_yjk2aIz4QHeM' 

d2g.upload(temp_df, spreadsheet_key, 'la_county_coronavirus', credentials=credentials, row_names=False)
d2g.upload(deaths_df, spreadsheet_key, 'case_deaths', credentials=credentials, row_names=False)

<Worksheet 'case_deaths' id:1930459353>

# Geocode

In [0]:
# prep location for geocoding. Explicitly add ', Los Angeles, California' so the geocoder finds the location

def geocode(df):
  df['loc_prep'] = 'temp'

  for i in range(len(df)):
    df['loc_prep'][i] = str(df.ocations[i]) + ', Los Angeles, California'

  # create and initialize coords column
  df['coords'] = 0.0

  # geocode locations
  geolocator = Nominatim(user_agent="sdfasdf")

  # to space out the geocoding
  from geopy.extra.rate_limiter import RateLimiter

  geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
  df['coords'] = df.loc_prep.apply(lambda x : geocode(x))

  #
  df['lat'] = df.coords.apply(lambda x: x.latitude if x !=None else 0.0)
  df['lon'] = df.coords.apply(lambda x: x.longitude if x !=None else 0.0)

  return df